In [1]:
import pandas as pd
import re
from utils.utils import save_json

In [2]:
df = []
data_dir = 'data_new'
# data_dir = 'data'
num_csv = 641 + 1
for i in range(num_csv):
    try:
        df.append(pd.read_csv(f'{data_dir}/car_info_requests_%06d.csv' % i))
    except:
        continue
df = pd.concat(df,axis=0)
df.dropna(axis=0,how='any',inplace=True)
print(f'number of sample = {len(df)}')
df.head(3)

number of sample = 35097


,change-speed,chepai-location,kilometer,price,shangpai-date,standard,title
0,自动,郑州,4.83万公里,20.68,2年10个月,国五,阿尔法·罗密欧-Stelvio 2018款 2.0T 200HP 精英版
1,自动,郑州,3.36万公里,21.88,2年10个月,国五,阿尔法·罗密欧-Stelvio 2018款 2.0T 200HP 精英版
2,自动,佛山,6.06万公里,19.98,2年11个月,国五,阿尔法·罗密欧-Stelvio 2018款 2.0T 200HP 精英版


In [3]:
def clean_title(df_ori):
    df = df_ori.copy()
    df.reset_index(inplace=True)
    df.loc[:,'title'] = df['title'].str.split('-',1,expand=True)[0]
    all_titles = df['title'].unique().tolist()
    title2idx = {
        title: idx for idx,title in enumerate(all_titles)
    }
    save_json(title2idx,f'{data_dir}/car_type_refinement.json')

    for index in df.index:
        df.loc[index,'title'] = title2idx[df.loc[index,'title']]
    df.loc[:,'title'] = df['title'].astype(int)
    return df

In [4]:
def clean_shangpai_date(df_ori):
    df = df_ori.copy()
    df.reset_index(inplace=True)
    # print(df.index)
    for index in df.index:
        shangpai = df.loc[index,'shangpai-date']
        # print(shangpai)
        ans = 0
        if "年" in shangpai:
            year = int(re.search(r'(\d+)年',shangpai).group()[0])
            ans += year
        if "月" in shangpai:
            month = int(re.search(r'(\d+)个月',shangpai).group()[0])
            ans += (month/12.0)
        df.loc[index,'shangpai-date'] = ans
    df.loc[:,'shangpai-date'] = df['shangpai-date'].astype(float)
    return df

In [5]:
def clean_change_speed(df_ori):
    df = df_ori.copy()
    df.loc[df['change-speed']=='自动','change-speed'] = 0
    df.loc[df['change-speed']=='手动','change-speed'] = 1
    df.loc[
        ~((df['change-speed']==0)|
        (df['change-speed']==1))
    ,'change-speed'] = 2
    df['change-speed'] = df['change-speed'].astype(int)
    return df

In [6]:
def clean_kilometer(df_ori):
    df = df_ori.copy()
    df.loc[:,'kilometer'] = df['kilometer'].str.split('万',1,expand=True)[0].astype(float)
    return df

In [7]:
def clean_price(df_ori):
    df = df_ori.copy()
    df.loc[:,'price'] = df['price'].astype(float)
    return df

In [8]:
def clean_standard(df_ori):
    df = df_ori.copy()
    conds = [
        df['standard']=='国一',
        df['standard']=='国二',
        df['standard']=='国三',
        df['standard']=='国四',
        df['standard']=='国五',
        df['standard']=='国六',
    ]
    tmp = conds[0]
    for i in range(1,len(conds)):
        tmp = (tmp) | (conds[i])
    tmp = ~tmp
    df.loc[tmp,'standard'] = 0

    for i in range(len(conds)):
        df.loc[conds[i],'standard'] = i+1
    df.loc[:,'standard'] = df['standard'].astype(int)
    return df

In [9]:
df = clean_kilometer(df)
df = clean_price(df)
df = clean_change_speed(df)
df = clean_shangpai_date(df)
df = clean_standard(df)
df = clean_title(df)
df.drop(['chepai-location'],axis=1,inplace=True)
df.drop(['index'],axis=1,inplace=True)
df.drop(['level_0'],axis=1,inplace=True)
df.head(4)

,change-speed,kilometer,price,shangpai-date,standard,title
0,0,4.83,20.68,2.083333,5,0
1,0,3.36,21.88,2.083333,5,0
2,0,6.06,19.98,2.083333,5,0
3,0,3.53,20.30,3.666667,5,0


In [10]:
df.to_csv(f'{data_dir}/data.csv',index=None)

In [11]:
num_all = len(df)
train = int(0.9*num_all)
df_shuffle = df.sample(frac=1.0)
df_train = df_shuffle.iloc[0:train,:].copy()
df_test = df_shuffle.iloc[train:,:].copy()
df_train.to_csv(f'{data_dir}/data_train.csv', index=None)
df_test.to_csv(f'{data_dir}/data_test.csv', index=None)